In [62]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from config import password

In [63]:
file = "aqi_full_data.csv"
air_quality_df = pd.read_csv(file, index_col=0)

file2 = "ca_sites.csv"
sites_df = pd.read_csv(file2, index_col=0)

file3 = "census_info.csv"
census_df = pd.read_csv(file3, index_col=0)

In [64]:
air_quality_df = air_quality_df.reset_index().rename(columns={'State Name':'State_Name', \
                                'county Name':'County', \
                                'Defining Parameter':'Defining_Parameter',\
                                'index':'aq_unique_no',\
                                'Defining Site':'site_no'})
air_quality_df.head()

,aq_unique_no,State_Name,County,Date,AQI,Category,Defining_Parameter,site_no
0,0,California,Alameda,2010-01-01,72,Moderate,PM2.5,06-001-0007
1,1,California,Alameda,2010-01-02,51,Moderate,PM2.5,06-001-2004
2,2,California,Alameda,2010-01-03,60,Moderate,PM2.5,06-001-0007
3,3,California,Alameda,2010-01-04,90,Moderate,PM2.5,06-001-0007
4,4,California,Alameda,2010-01-05,90,Moderate,PM2.5,06-001-0011


In [51]:
sites_table_df = sites_df.reset_index().rename(columns={'Defining Site':'site_no','Land Use':'Land_Use', \
                                'Location Setting':'Location_Setting', \
                                'State Name':'State_Name', \
                                'County Code':'County_Code', \
                                'City Name':'City_Name', \
                                'CBSA Name':'CBSA_Name'})\
                    .drop(columns=['County Name'])
                    

# site_count = sites_df.groupby('site_no')
# site_unique = site_count['County Name'].unique().count()
# print(site_unique)
sites_table_df.head()

,site_no,Latitude,Longitude,Elevation,Land_Use,Location_Setting,State_Name,County_Code,City_Name,CBSA_Name
0,06-001-0001,37.876870,-122.266913,15.0,RESIDENTIAL,URBAN AND CENTER CITY,California,1,Berkeley,"San Francisco-Oakland-Hayward, CA"
1,06-001-0002,37.533243,-121.958813,22.0,COMMERCIAL,URBAN AND CENTER CITY,California,1,Fremont,"San Francisco-Oakland-Hayward, CA"
2,06-001-0003,37.684900,-121.765900,150.0,COMMERCIAL,URBAN AND CENTER CITY,California,1,Livermore,"San Francisco-Oakland-Hayward, CA"
3,06-001-0004,37.800484,-122.266358,38.0,COMMERCIAL,URBAN AND CENTER CITY,California,1,Oakland,"San Francisco-Oakland-Hayward, CA"
4,06-001-0005,37.798600,-122.268400,10.0,COMMERCIAL,URBAN AND CENTER CITY,California,1,Oakland,"San Francisco-Oakland-Hayward, CA"


In [52]:
census_df = census_df.dropna().apply(lambda x: x.str.strip() if x.dtype == "object" else x)
census_count = census_df.dropna().groupby('County')

census_unique = census_count['County'].unique().count()
print(census_unique)
census_df.head()

58


,County,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,Alameda,1510271,1510258,1512986,1530915,1553764,1579593,1607792,1634538,1650950,1660196,1666756,1671329
2,Alpine,1175,1175,1161,1093,1110,1128,1080,1077,1047,1111,1089,1129
3,Amador,38091,38091,37886,37543,37104,36620,36726,37031,37429,38529,39405,39752
4,Butte,220000,220005,219949,219975,220869,221641,223516,224631,226231,228696,230339,219186
5,Calaveras,45578,45578,45468,45160,44815,44655,44671,44965,45322,45681,45698,45905


In [53]:
county_name = sites_df.groupby('County Code')
county_name_unique = county_name['County Name'].first()
county_code_unique = county_name['County Name'].unique().count()

county_df = pd.DataFrame({'County':county_name_unique}).rename_axis('county_code')
county_table_df = county_df.reset_index().rename(columns={'County':'county_name'})
county_table_df.head()

,county_code,county_name
0,1,Alameda
1,3,Alpine
2,5,Amador
3,7,Butte
4,9,Calaveras


In [54]:
county_code_census = county_df.reset_index().merge(census_df,on='County')
county_code_census.head()

,county_code,County,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1,Alameda,1510271,1510258,1512986,1530915,1553764,1579593,1607792,1634538,1650950,1660196,1666756,1671329
1,3,Alpine,1175,1175,1161,1093,1110,1128,1080,1077,1047,1111,1089,1129
2,5,Amador,38091,38091,37886,37543,37104,36620,36726,37031,37429,38529,39405,39752
3,7,Butte,220000,220005,219949,219975,220869,221641,223516,224631,226231,228696,230339,219186
4,9,Calaveras,45578,45578,45468,45160,44815,44655,44671,44965,45322,45681,45698,45905


In [67]:
air_quality_table_df = county_df.reset_index().merge(air_quality_df,on='County').drop(columns=['County'])
air_quality_table_df.head()

,county_code,aq_unique_no,State_Name,Date,AQI,Category,Defining_Parameter,site_no
0,1,0,California,2010-01-01,72,Moderate,PM2.5,06-001-0007
1,1,1,California,2010-01-02,51,Moderate,PM2.5,06-001-2004
2,1,2,California,2010-01-03,60,Moderate,PM2.5,06-001-0007
3,1,3,California,2010-01-04,90,Moderate,PM2.5,06-001-0007
4,1,4,California,2010-01-05,90,Moderate,PM2.5,06-001-0011


In [55]:
county_code_census = county_code_census.drop(columns=['County', 'Census','Estimates Base']).set_index('county_code')
county_code_census.head()

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
county_code,,,,,,,,,,
1,1512986,1530915,1553764,1579593,1607792,1634538,1650950,1660196,1666756,1671329
3,1161,1093,1110,1128,1080,1077,1047,1111,1089,1129
5,37886,37543,37104,36620,36726,37031,37429,38529,39405,39752
7,219949,219975,220869,221641,223516,224631,226231,228696,230339,219186
9,45468,45160,44815,44655,44671,44965,45322,45681,45698,45905


In [56]:
county_code_census_pop_year = county_code_census.stack().reset_index()\
                                                .rename(columns={'level_1':'year',0:'population'})\
                                                .rename_axis('census_unique_no').reset_index()
county_code_census_pop_year

,census_unique_no,county_code,year,population
0,0,1,2010,1512986
1,1,1,2011,1530915
2,2,1,2012,1553764
3,3,1,2013,1579593
4,4,1,2014,1607792
...,...,...,...,...
575,575,115,2015,74045
576,576,115,2016,74952
577,577,115,2017,76578
578,578,115,2018,77557


In [57]:
# create connection to ETL_project_DB in postgres
engine = create_engine('postgresql://postgres:'+ password + '@localhost:5432/AirQuality_DB')
connection = engine.connect()

In [68]:
# check table names in database
engine.table_names()

['County', 'Sites', 'AirQuality', 'CensusPopulation']

In [69]:
# load dataframes into postgres using pandas
county_table_df.to_sql(name='County', con=engine, if_exists='append', index=False)

county_code_census_pop_year.to_sql(name='CensusPopulation', con=engine, if_exists='append', index=False)

sites_table_df.to_sql(name='Sites', con=engine, if_exists='append', index=False)

air_quality_table_df.to_sql(name='AirQuality', con=engine, if_exists='append', index=False)


